In [63]:
import pandas as pd
from pathlib import Path
import numpy as np

# Folder containing the 5 NGFS Phase-5 Excel files
# Example structure (adjust if your folder name is different):
# Datasets/NGFS-Phase-5/
ngfs_folder = Path("Datasets") / "NGFS-Phase-5"

ngfs_files = sorted(list(ngfs_folder.glob("*.xlsx")))
print(f"Found {len(ngfs_files)} NGFS files:")
for f in ngfs_files:
    print(" -", f.name)

Found 5 NGFS files:
 - Downscaled_GCAM 6.0 NGFS_data.xlsx
 - Downscaled_MESSAGEix-GLOBIOM 2.0-M-R12-NGFS_data.xlsx
 - Downscaled_REMIND-MAgPIE 3.3-4.8_data.xlsx
 - IAM_data.xlsx
 - NiGEM_data.xlsx


In [53]:
def clean_cols(df):
    df = df.copy()
    df.columns = (
        df.columns.astype(str)
        .str.strip()
        .str.replace("\n", " ")
        .str.replace("  ", " ")
        .str.replace("Unnamed.*", "", regex=True)
    )
    return df

In [54]:
def preprocess_ngfs(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df = df.replace("", pd.NA)          # empty → NaN
    df = df.dropna(axis=1, how="all")   # remove empty cols
    df = df.dropna(axis=0, how="all")   # remove empty rows
    df = df.reset_index(drop=True)
    return df

In [55]:
ngfs_frames = []
total_rows = 0

for f in ngfs_files:
    print(f"\nReading: {f.name}")
    df_raw = pd.read_excel(f)
    print("  Original shape:", df_raw.shape)
    df_raw = clean_cols(df_raw)
    df_raw = preprocess_ngfs(df_raw)
    print("  After preprocess:", df_raw.shape)

    df_raw["Provider"] = "NGFS-Phase-5"
    ngfs_frames.append(df_raw)
    total_rows += len(df_raw)


Reading: Downscaled_GCAM 6.0 NGFS_data.xlsx
  Original shape: (212649, 86)
  After preprocess: (212649, 86)

Reading: Downscaled_MESSAGEix-GLOBIOM 2.0-M-R12-NGFS_data.xlsx
  Original shape: (175203, 86)
  After preprocess: (175203, 86)

Reading: Downscaled_REMIND-MAgPIE 3.3-4.8_data.xlsx
  Original shape: (388219, 86)
  After preprocess: (388219, 86)

Reading: IAM_data.xlsx
  Original shape: (404631, 22)
  After preprocess: (404631, 22)

Reading: NiGEM_data.xlsx
  Original shape: (100776, 34)
  After preprocess: (100776, 34)


In [115]:
# Merge all rows from all files
ngfs_data = pd.concat(ngfs_frames, axis=0, ignore_index=True)
print("\nExpected total rows (sum of each file):", total_rows)
print("Actual merged rows:", len(ngfs_data))
print("Merged shape:", ngfs_data.shape)
ngfs_data.tail(5)


Expected total rows (sum of each file): 1281478
Actual merged rows: 1281478
Merged shape: (1281478, 87)


,Model,Scenario,Region,Variable,Unit,2020,2021,2022,2023,2024,...,2092,2093,2094,2095,2096,2097,2098,2099,2100,Provider
1281473,NiGEM NGFS v1.24.2[REMIND-MAgPIE 3.3-4.8],Below 2°C,NiGEM NGFS v1.24.2|World,Quarterly consumption of non-carbon ; MnToe(ph...,% difference,NaN,NaN,0.0,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NGFS-Phase-5
1281474,NiGEM NGFS v1.24.2[REMIND-MAgPIE 3.3-4.8],Below 2°C,NiGEM NGFS v1.24.2|World,Quarterly consumption of non-carbon ; MnToe(tr...,% difference,NaN,NaN,0.0,1.493696,2.000135,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NGFS-Phase-5
1281475,NiGEM NGFS v1.24.2[REMIND-MAgPIE 3.3-4.8],Below 2°C,NiGEM NGFS v1.24.2|World,Quarterly consumption of oil ; MnToe(combined),% difference,NaN,NaN,0.0,-1.562753,-2.128934,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NGFS-Phase-5
1281476,NiGEM NGFS v1.24.2[REMIND-MAgPIE 3.3-4.8],Below 2°C,NiGEM NGFS v1.24.2|World,Quarterly consumption of oil ; MnToe(physical),% difference,NaN,NaN,0.0,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NGFS-Phase-5
1281477,NiGEM NGFS v1.24.2[REMIND-MAgPIE 3.3-4.8],Below 2°C,NiGEM NGFS v1.24.2|World,Quarterly consumption of oil ; MnToe(transition),% difference,NaN,NaN,0.0,-1.562753,-2.128934,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NGFS-Phase-5


In [118]:
# 1. Detect year columns
year_cols = [col for col in ngfs_data.columns if str(col).isdigit()]

# 2. Convert to integers for modulo operations
year_cols_int = list(map(int, year_cols))

# 3. Keep only every 5-year interval
five_years = [y for y in year_cols_int if y % 5 == 0]
five_year_cols = [str(y) for y in five_years if str(y) in ngfs_data.columns]

# 4. Metadata columns to keep
meta_cols = ["Model", "Scenario", "Region", "Variable", "Unit", "Provider"]

# 5. Create a SAFE copy → prevents SettingWithCopyWarning
ngfs_5yr = ngfs_data[meta_cols + five_year_cols].copy()

# 6. Convert numeric year columns safely
ngfs_5yr[five_year_cols] = ngfs_5yr[five_year_cols].apply(
    lambda col: pd.to_numeric(col, errors="coerce")
)

print("Original shape:", ngfs_data.shape)
print("5-year shape:", ngfs_5yr.shape)
ngfs_5yr.head()

Original shape: (1156624, 23)
5-year shape: (1156624, 23)


,Model,Scenario,Region,Variable,Unit,Provider,2020,2025,2030,2035,...,2055,2060,2065,2070,2075,2080,2085,2090,2095,2100
0,Downscaling[GCAM 6.0 NGFS],Delayed transition,AGO,Carbon Sequestration|CCS,Mt CO2/yr,NGFS-Phase-5,0.0000,0.0006,0.0064,0.2977,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Downscaling[GCAM 6.0 NGFS],Delayed transition,AGO,Carbon Sequestration|CCS|Biomass,Mt CO2/yr,NGFS-Phase-5,0.0000,0.0002,0.0049,0.0972,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Downscaling[GCAM 6.0 NGFS],Delayed transition,AGO,Carbon Sequestration|CCS|Fossil,Mt CO2/yr,NGFS-Phase-5,0.0000,0.0004,0.0015,0.0986,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Downscaling[GCAM 6.0 NGFS],Delayed transition,AGO,Carbon Sequestration|CCS|Industrial Processes,Mt CO2/yr,NGFS-Phase-5,0.0000,0.0000,0.0000,0.1019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Downscaling[GCAM 6.0 NGFS],Delayed transition,AGO,Emissions|CO2,Mt CO2/yr,NGFS-Phase-5,148.1421,90.3493,129.7740,156.4094,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [119]:
# Convert year columns to numeric
year_cols = [col for col in ngfs_5yr.columns if str(col).isdigit()]
for col in year_cols:
    ngfs_5yr[col] = pd.to_numeric(ngfs_5yr[col], errors="coerce")

# Detect rows where ALL years are 0 or NaN
mask_all_zero = (ngfs_5yr[year_cols].fillna(0) == 0).all(axis=1)
zero_rows_df = ngfs_5yr[mask_all_zero].copy()
output_zero_csv = "Datasets/processed/NGFS-Phase-5/NGFS_zero_year_rows.csv"
zero_rows_df.to_csv(output_zero_csv, index=False)

print("Rows where all years are zero or NaN:", mask_all_zero.sum())

# Remove such rows
ngfs_data = ngfs_5yr[~mask_all_zero].reset_index(drop=True)

print("New shape:", ngfs_data.shape)

Rows where all years are zero or NaN: 0
New shape: (1156624, 23)


In [120]:
# Save merged NGFS dataset (still wide format)
output_path = Path("Datasets/processed/NGFS-Phase-5/main/NGFS_Phase-5.csv")
ngfs_data.to_csv(output_path, index=False)
output_path

# ---------------------------------------------------------------------------------------------------------------------------------------

WindowsPath('Datasets/processed/NGFS-Phase-5/Merged_NGFS_Phase-5.csv')

In [124]:
import pandas as pd

df = pd.read_csv("Datasets/processed/main/IPCC_AR6_Scenarios_Database_ISO.csv")
df["Provider"] = "IPCC-AR6"
len(df.shape)


2

In [122]:
ngfs_data.shape

(1156624, 23)